<a href="https://colab.research.google.com/github/strategiepilot/ML_workshop/blob/master/ML_Einfuehrung%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import os
path = "/gdrive/My Drive/data"
os.chdir(path)
os.getcwd()
os.listdir()


['TIMEGenderData.csv',
 'Kopie von iris.csv',
 'times_magazin.csv',
 'noten.csv',
 'cs-Data Dictionary.xls',
 'cs-sampleEntry.csv',
 'cs-test.csv',
 'bmw_pricing_challenge.csv',
 'cs-training.csv',
 'cars.csv']

Damit fangen wir immer an ....

In [0]:
%matplotlib inline

# Grundausstattung an Bibliotheken, die wir immer laden
import numpy as np                  # Numerische Operationen, Lineare Algebra
from scipy.stats import *           # Funktionsbibliothek mit statistischen Funktionen
import matplotlib.pyplot as plt     # Visualisierung von Daten/Ergenissen
import pandas as pd                 # Bearbeitung von Tabellen (sog. Data Frames)
import seaborn as sns               # Erweiterte Visualisierung von Daten/Ergebnissen etc.
import warnings                     # Ermöglicht die Deaktivierung von best. Warnmeldungen
import random                       # Damit kann man Zufallszahlen generieren
import os                           # Ermöglicht Zugriff auf das Dateiablagesystem 

# Ein paar Einstellungen, die einem das Leben einfacher machen
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [8, 4]
from IPython.core.pylabtools import figsize
plt.style.use('seaborn-white')
sns.set_style('white')
sns.set_context('talk')

path = "/content/drive/My Drive/data/"

### 3.3 Supervised Learning: Classification

#### Give Me Some Credit
https://www.kaggle.com/c/GiveMeSomeCredit/data



In [4]:
os.chdir("/gdrive/My Drive/data")

cs = pd.read_csv("cs-training.csv")
cs.iloc[:,0].value_counts(normalize=True)
cs = cs.iloc[:,1:]

cs.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 12.6 MB


In [5]:
cs["MonthlyIncome"] = cs["MonthlyIncome"].fillna( cs["MonthlyIncome"].median() )
cs["NumberOfDependents"] = cs["NumberOfDependents"].fillna( cs["NumberOfDependents"].median() )
cs.isna().sum()

SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

In [6]:
cs2 = cs.copy()        # Target Variable wird nicht 

cs2 = cs.iloc[:,1:]        # Target Variable wird nicht 

# data = cs2.iloc[:,1:]

cs2 = cs2[np.abs(cs2-cs2.mean()) <= (3*cs2.std())]  # schneidet die Ränder ab alles was außerhalb von 3 Standardabw. liegt
cs2 = cs2.merge(cs.SeriousDlqin2yrs, left_index=True, right_index=True, )
cs2 = cs2.fillna( cs2.median())
# cs2 = cs2[np.abs(cs2-cs2.mean()) <= (3*cs2.std())]  # schneidet die Ränder ab alles was außerhalb von 3 Standardabw. liegt

cs2.describe().T

,count,mean,std,min,25%,50%,75%,max
RevolvingUtilizationOfUnsecuredLines,150000.0,0.420734,7.041555,0.0,0.029867,0.15371,0.555779,747.0
age,150000.0,52.281313,14.748528,21.0,41.000000,52.00000,63.000000,96.0
NumberOfTime30-59DaysPastDueNotWorse,150000.0,0.245267,0.696454,0.0,0.000000,0.00000,0.000000,12.0
DebtRatio,150000.0,298.125877,873.165950,0.0,0.175074,0.36458,0.837266,6466.0
MonthlyIncome,150000.0,6155.775693,4069.116628,0.0,3903.000000,5400.00000,7333.000000,45000.0
NumberOfOpenCreditLinesAndLoans,150000.0,8.201547,4.620626,0.0,5.000000,8.00000,11.000000,23.0
NumberOfTimes90DaysLate,150000.0,0.089447,0.472688,0.0,0.000000,0.00000,0.000000,12.0
NumberRealEstateLoansOrLines,150000.0,0.962960,0.938918,0.0,0.000000,1.00000,2.000000,4.0
NumberOfTime60-89DaysPastDueNotWorse,150000.0,0.064707,0.329788,0.0,0.000000,0.00000,0.000000,11.0
NumberOfDependents,150000.0,0.701713,1.038216,0.0,0.000000,0.00000,1.000000,4.0


In [0]:
cs3.to_csv("cs-training-small.csv")

In [9]:
cs2.SeriousDlqin2yrs.value_counts(normalize=True)

0    0.93316
1    0.06684
Name: SeriousDlqin2yrs, dtype: float64

In [11]:
cs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null float64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null float64
DebtRatio                               150000 non-null float64
MonthlyIncome                           150000 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null float64
NumberOfTimes90DaysLate                 150000 non-null float64
NumberRealEstateLoansOrLines            150000 non-null float64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null float64
NumberOfDependents                      150000 non-null float64
SeriousDlqin2yrs                        150000 non-null int64
dtypes: float64(10), int64(1)
memory usage: 12.6 MB


In [0]:
from sklearn.model_selection import train_test_split
X = cs2.iloc[:,:-1]
y = cs2.iloc[:,-1]

X_small, X_large, y_small, y_large = train_test_split(X, y, test_size=0.7, shuffle=True, stratify=y, random_state=123)

In [14]:
X_small.shape, y_small.shape
y_small.value_counts(normalize=True)

0    0.933156
1    0.066844
Name: SeriousDlqin2yrs, dtype: float64

In [23]:
cs3 = pd.concat([X_small, y_small], axis=1)
cs3.reset_index( inplace=True, drop=True)
cs3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 11 columns):
RevolvingUtilizationOfUnsecuredLines    45000 non-null float64
age                                     45000 non-null float64
NumberOfTime30-59DaysPastDueNotWorse    45000 non-null float64
DebtRatio                               45000 non-null float64
MonthlyIncome                           45000 non-null float64
NumberOfOpenCreditLinesAndLoans         45000 non-null float64
NumberOfTimes90DaysLate                 45000 non-null float64
NumberRealEstateLoansOrLines            45000 non-null float64
NumberOfTime60-89DaysPastDueNotWorse    45000 non-null float64
NumberOfDependents                      45000 non-null float64
SeriousDlqin2yrs                        45000 non-null int64
dtypes: float64(10), int64(1)
memory usage: 3.8 MB


In [29]:
cs3.describe(percentiles=[.05,.25,.5,.75,.95], ).T

,count,mean,std,min,5%,25%,50%,75%,95%,max
RevolvingUtilizationOfUnsecuredLines,45000.0,0.416316,6.662989,0.0,0.000000,0.030052,0.154618,0.554526,1.0,658.0
age,45000.0,52.203289,14.769210,21.0,29.000000,41.000000,52.000000,63.000000,78.0,96.0
NumberOfTime30-59DaysPastDueNotWorse,45000.0,0.248778,0.704626,0.0,0.000000,0.000000,0.000000,0.000000,2.0,12.0
DebtRatio,45000.0,301.823800,878.506793,0.0,0.004497,0.177000,0.366250,0.856588,2334.1,6466.0
MonthlyIncome,45000.0,6116.426533,4022.650588,0.0,1500.000000,3878.000000,5400.000000,7254.500000,13058.4,45000.0
NumberOfOpenCreditLinesAndLoans,45000.0,8.208067,4.620116,0.0,2.000000,5.000000,8.000000,11.000000,17.0,23.0
NumberOfTimes90DaysLate,45000.0,0.088733,0.473355,0.0,0.000000,0.000000,0.000000,0.000000,1.0,11.0
NumberRealEstateLoansOrLines,45000.0,0.967444,0.940120,0.0,0.000000,0.000000,1.000000,2.000000,3.0,4.0
NumberOfTime60-89DaysPastDueNotWorse,45000.0,0.064933,0.334411,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.0
NumberOfDependents,45000.0,0.701756,1.036654,0.0,0.000000,0.000000,0.000000,1.000000,3.0,4.0


In [37]:
from sklearn.model_selection import train_test_split

X = cs3.iloc[:,:-1]
y = cs3.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33333, shuffle=True, stratify=y, random_state=123)
print(f' Trainingsset: {X_train.shape, y_train.shape} / Test Set: {X_test.shape, y_test.shape}')

 Trainingsset: ((30000, 10), (30000,)) / Test Set: ((15000, 10), (15000,))


In [46]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import time

# Hier sind unsere Kandidaten
clf1 = GaussianNB()
clf2 = SVC(class_weight="balanced",)
clf3 = LogisticRegression(class_weight="balanced")
clf4 = KNeighborsClassifier( )
# Dem Random Forest spendieren wir 3 Varianten ...
clf5 = RandomForestClassifier(class_weight="balanced", n_jobs=-1)
clf6 = RandomForestClassifier(n_estimators = 300, class_weight="balanced", max_depth=3,  bootstrap=True, n_jobs=-1)
clf7 = RandomForestClassifier(n_estimators = 500, class_weight="balanced", max_depth=5,  bootstrap=False, n_jobs=-1)

# Wir speichern die "Rundenergebnisse" der einzelnen Classifier in einem Dataframe
results = pd.DataFrame( {"Estimator":[], "Accuracy":[], "Precision":[], "Recall":[], "f1":[], "AUC":[], "Duration":[]} )
models_fitted = []  # Ablegen der gefitteten Modelle (Objekte) in einer Liste

# Das ist unsere Pipeline die wir durchlaufen
Listing = [(1, "NB",clf1),
           #(2, "SVC", clf2),
           (3, "LogR", clf3),
           (4, "Knn5", clf4),
           (5, "RF", clf5),
           (6, "RF opt1", clf6),
           (7, "RF opt2", clf7),
          ]  

# Durchlauf mehrerer Modelle und Wegschreiben des Ergebnisses

for i, name, estimator in Listing:
    
    # Model fitten u. in Liste ablegen
    start = time.time()                     # Stoppuhr: Zwischenzeit nehmen
    est = estimator.fit(X_train, y_train)   # model aus Listing nehmen und fitten
    models_fitted.append(est)

    # Scorings erstellen
    ytrue = y_test                          # ...
    ypred = est.predict(X_test)             # model auf Testdaten anwenden (predict)
    
    acc = accuracy_score(ytrue, ypred )     # Accuracy 
    prec = precision_score(ytrue, ypred )   # Precision 
    rec = recall_score(ytrue, ypred,  )     # Recall
    f1 = f1_score(ytrue, ypred, )           # f1-Score
    auc = roc_auc_score(ytrue, ypred, )     # AUC
    end = time.time()                       # Stoppuhr: Zwischenzeit nehmen
    duration = end - start                  # Walltime in Variable abspeichern
    
    results.loc[i,:] = [name, acc, prec, rec, f1, auc, duration]
    
print(f"Baseline Accuracy: {1-y_test.mean()}")
results.round(3)

Baseline Accuracy: 0.9331333333333334


,Estimator,Accuracy,Precision,Recall,f1,AUC,Duration
1,NB,0.902,0.346,0.516,0.415,0.723,0.030
3,LogR,0.826,0.222,0.638,0.330,0.739,0.503
4,Knn5,0.932,0.319,0.015,0.029,0.506,0.615
5,RF,0.935,0.549,0.152,0.238,0.571,3.270
6,RF opt1,0.775,0.196,0.764,0.312,0.770,3.040
7,RF opt2,0.793,0.211,0.767,0.331,0.781,9.225


In [47]:
models_fitted

[GaussianNB(priors=None, var_smoothing=1e-09),
 LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                      weights='uniform'),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estim

# Parkplatz

In [0]:
## Home Credit Datenset verkleinern auf 30k Datensätze (10%)
hc = pd.read_csv("application_train.csv") 
X = hc.iloc[:,2:].copy()
y = hc.iloc[:,1].copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, shuffle=True, stratify=y, random_state=42)

print(f' Trainingsset: {X_train.shape, y_train.shape}  / Test Set: {X_test.shape, y_test.shape}')
print(y_train.value_counts(normalize=True))

X_train.index.name = "idx"
y_train.index.name = "idx"
X2 = X_train.merge(y_train, on="idx")
X2.shape


In [0]:
X2.to_csv("home_credit_small.csv")

In [0]:
mtcars = pd.read_csv("mtcars.csv", index_col=0, sep=";", decimal=",")
mtcars = mtcars.fillna(mtcars.mean())
mtcars.colour = mtcars.colour.fillna("MISSING")

In [0]:
# Standardize data (0 mean, 1 stdev)
from sklearn.preprocessing import StandardScaler
import pandas
import numpy
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'klass']
dataframe = pandas.read_csv(url, names=names, )
dataframe.index.name="idx"
array = dataframe.values
print(dataframe.shape)
# separate array into input and output components
X = array[:,0:8]
Y = array[:,8]
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)
# summarize transformed data
numpy.set_printoptions(precision=3)
print(rescaledX[0:5,:])

df = pd.DataFrame(rescaledX, columns=names[:8])
df.index.name="idx"
df = df.merge(dataframe.klass, on="idx" )
print(df.shape)
df


In [0]:
dataframe.klass

Variablen im Boston Housing Dataset:
+ CRIM - per capita crime rate by town
+ ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
+ INDUS - proportion of non-retail business acres per town.
+ CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
+ NOX - nitric oxides concentration (parts per 10 million)
+ RM - average number of rooms per dwelling
+ AGE - proportion of owner-occupied units built prior to 1940
+ DIS - weighted distances to five Boston employment centres
+ RAD - index of accessibility to radial highways
+ TAX - full-value property-tax rate per USD 10,000
+ PTRATIO - pupil-teacher ratio by town
+ B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
+ LSTAT - % lower status of the population
+ MEDV - Median value of owner-occupied homes in USD 1000's

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test_total, y_train, y_test_total = train_test_split(X, y, test_size=0.4, shuffle=True, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_total, y_test_total, test_size=0.5, shuffle=True, stratify=y_test_total, random_state=42)
print(f' Trainingsset: {X_train.shape, y_train.shape} / Validation Set: {X_val.shape, y_val.shape}  / Test Set: {X_test.shape, y_test.shape}')

In [0]:
# Confusion Matix plotten
def Plot_confMatrix(y_real, y_pred, title='Titel'):
    '''
    Erstellen einer Confusion Matrix Grafik
    im Abgleich von Label y_real und Prognose y_pred
    '''
    from sklearn.metrics import confusion_matrix
    mat = confusion_matrix(y_real, y_pred)
    sns.set(font_scale=1.4)
    sns.heatmap(mat, square=True, annot=True, fmt='d',  cmap='Blues', cbar=True, )
    plt.xlabel('Prediction')
    plt.ylabel('True value')
    plt.title(title);
    return plt.show()   

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
## Home Credit Datenset verkleinern auf 30k Datensätze (10%)
hc = pd.read_csv("application_train.csv") 
X = hc.iloc[:,2:].copy()
y = hc.iloc[:,1].copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, shuffle=True, stratify=y, random_state=42)

print(f' Trainingsset: {X_train.shape, y_train.shape}  / Test Set: {X_test.shape, y_test.shape}')
print(y_train.value_counts(normalize=True))

X.index.name = "idx"
y.index.name = "idx"
X2 = X.merge(y, on="idx")



In [0]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

model = 'Baseline Decision Tree'
t_names = ['Manuelle Entscheidung', 'Systementscheidung']

estimator = DecisionTreeClassifier(max_depth=5)
estimator.fit(X_train, y_train)

ytrue = y_val
ypred = estimator.predict(X_val)
ppred = estimator.predict_proba(X_val)
ppred_1 = ppred[:, 1]
ppred_0 = ppred[:, 0]

accuracy = accuracy_score(ytrue, ypred)
roc_auc = roc_auc_score(ytrue, ppred_1)
print(f'Accuracy Score: {accuracy:.4f}, AUC: {roc_auc:.4f}')
print(classification_report(ytrue, ypred, target_names=t_names))

# Feature Importance aus Model in Dataframe FI schreiben
fi_data = {'Feature': list(X_train.columns), 'F_Importance': estimator.feature_importances_}
FI = pd.DataFrame(data=fi_data)
FI = FI.sort_values('F_Importance', ascending=False)[:15]
FI

In [0]:
os.chdir("F://Data/Data Science Uni/40200/BMW Bank Seminar/Data Sets")
os.listdir()

In [0]:
cars.columns

In [0]:
cols = [c.strip() for c in cars.columns]

In [0]:
cars.info()

In [0]:
cars.describe()

In [0]:
os.chdir("F://Data/Data Science Uni/40200/BMW Bank Seminar/Data Sets")
os.listdir()

cars = pd.read_csv("cars.csv", sep=",", decimal=".")

# Leerstellen in Bezeichnungen entfernen und falsche Datentypen korrigieren
cols = [c.strip() for c in cars.columns]
cars.columns = cols
cars.country = cars.country.str.strip()
cars.cubicinches = cars.cubicinches.str.strip()
cars.weightlbs = cars.weightlbs.str.strip()
cars.cubicinches = pd.to_numeric(cars.cubicinches)
cars.weightlbs = pd.to_numeric(cars.weightlbs)

# Spalten mit NA-Werten identifizieren und mit Mittelwert imputieren
cols_nan = [c for c in cars.columns if any(cars[c].isna())]
cars[cols_nan] = cars[cols_nan].fillna(cars[cols_nan].mean())

# Neue Spalte "country_code": Country (Datentyp Text) in numerisches Format encoden
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cars["country_code"] = pd.Series(le.fit_transform(cars.country))

print(list(enumerate(le.classes_)))
cars.head()


In [0]:
sns.pairplot(cars, size=2)

In [0]:

# k-Means Algorithmus anwenden
from sklearn.cluster import KMeans

X = cars.drop(["country", "country_code"], axis=1)
y = cars.country_code

km = KMeans(n_clusters=3).fit(X)           # Wendet k-Means auf X an, mit Vorgabe 3 Cluster
km.labels_                                 # k-Means hat für jeden Datenpunkt eine Zuordnung vorgenommen

In [0]:
# Visualisierung der Zuordnungen
figsize(12,8)
print(list(enumerate(X.columns)))

i,j = 0,6

plt.scatter(X.iloc[:,i], X.iloc[:,j], c=km.labels_, cmap="rainbow")
plt.scatter(km.cluster_centers_[:,i], km.cluster_centers_[:,j], c='k', marker='*', s=300, ) # Scatterplot mit Centroids
plt.show()

Da wir das Modell jetzt trainiert haben, können wir es verwenden, um NEUE Datensätze zu beurteilen:

In [0]:
# Wir pflücken drei neue Schwertlilien, vermessen Ihre Blätter... 
iris_New = np.array([[5, 2.4, 6.2, 1.8],
                     [3, 3, 4, 3],
                     [4.2, 7, 4, 1.2]])

#  und nutzen jetzt unser trainiertes Modell km zur Einordnung
km.predict(iris_New)